### Test Downloads

In [8]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from data_pipeline.src import downloader, config, load_anndata, combine, dataloader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
downloader.run_downloads()


Saving extracted PBMC datasets to: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw

✓ Already downloaded/extracted: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw\CD34
------------------------------------------------------------
✓ Already downloaded/extracted: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw\CD19_B
------------------------------------------------------------
✓ Already downloaded/extracted: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw\CD56_NK
------------------------------------------------------------
✓ Already downloaded/extracted: C:\Users\33699\Docum

In [5]:
folder = Path("data_pipeline/data/pbmc_raw/CD4_CD45RO")
meta = config.DATASETS["CD4_CD45RO"]

adata = load_anndata.load_anndata(folder, meta)
adata

Loading 10x data from: data_pipeline\data\pbmc_raw\CD4_CD45RO\filtered_matrices_mex\hg19
✓ Loaded 10224 cells × 32738 genes from CD4_CD45RO


AnnData object with n_obs × n_vars = 10224 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'

In [7]:
combined, collection = combine.run_combine(
    do_write_shards=True,
    write_combined=True,
    harmonize_var=False
)

Loading all PBMC AnnData objects...
📦 Loading dataset: CD34
Loading 10x data from: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw\CD34\filtered_matrices_mex\hg19
✓ Loaded 9232 cells × 32738 genes from CD34
📦 Loading dataset: CD19_B
Loading 10x data from: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw\CD19_B\filtered_matrices_mex\hg19
✓ Loaded 10085 cells × 32738 genes from CD19_B
📦 Loading dataset: CD56_NK
Loading 10x data from: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_raw\CD56_NK\filtered_matrices_mex\hg19
✓ Loaded 8385 cells × 32738 genes from CD56_NK
📦 Loading dataset: CD4_helper
Loading 10x data from: C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphic

c:\Users\33699\Python\venv\ds\Lib\site-packages\anndata\_core\anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


✓ Combined AnnData written to C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_processed\pbmc_combined.h5ad
✓ Label maps saved to C:\Users\33699\Documents\Ressources\Cours\MVA\Probabilistic graphical models and deep generative models\PROJET\scVAE_mva2025\data_pipeline\data\pbmc_processed\label_maps.json
Building in‑memory AnnCollection...


c:\Users\33699\Python\venv\ds\Lib\site-packages\legacy_api_wrap\__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ AnnCollection constructed from 9 shards.

Combine stage complete!



In [9]:
import scanpy as sc
adata_combined = sc.read_h5ad("data_pipeline/data/pbmc_processed/pbmc_combined.h5ad")
adata_combined

c:\Users\33699\Python\venv\ds\Lib\site-packages\anndata\_core\anndata.py:1796: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


AnnData object with n_obs × n_vars = 92043 × 32738
    obs: 'cell_type_lvl1', 'cell_type_lvl2', 'cell_type_lvl3', 'cell_type_lvl4', 'dataset', 'cell_type_lvl1__code', 'cell_type_lvl2__code', 'cell_type_lvl3__code', 'cell_type_lvl4__code'
    var: 'gene_ids'
    uns: 'label_maps'

In [10]:
train_loader, val_loader, test_loader = dataloader.build_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=512,
    one_hot=True,      
)

for i, batch in enumerate(train_loader):
        print("Example batch shapes:")
        print("X:", batch["X"].shape, "y1:", batch["y1"].shape, "y4:", batch["y4"].shape)
        break

c:\Users\33699\Python\venv\ds\Lib\site-packages\legacy_api_wrap\__init__.py:88: UserWarning: Observation names are not unique.
  return fn(*args_all, **kw)


✓ Built AnnCollection with 92043 total cells.
Split: train=74554, val=8283, test=9206
✓ Train=74554, Val=8283, Test=9206
Example batch shapes:
X: torch.Size([512, 1, 32738]) y1: torch.Size([512, 1, 2]) y4: torch.Size([512, 1, 9])


In [22]:
%load_ext autoreload
%autoreload 2

from mixture_vae.mvae import ind_MoMVAE

from mixture_vae.distributions import (
    NormalDistribution,
    UniformDistribution,
    NegativeBinomial
)

from mixture_vae.training import (
    training_mvae
)

import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
model = ind_MoMVAE(
    [
        {
          "input_dim": batch["X"].shape[-1],
          "hidden_dim": 20,
          "n_components": K,
          "n_layers": 3,
          "norm_layer": torch.nn.Identity,
          "prior_latent": NormalDistribution(
              {
              "mu": torch.zeros((1, 20)),
              "std": torch.ones((1, 20))
              }
          ),
          "prior_input": NegativeBinomial(
              {
                "p":  0.5 * torch.ones((1, batch["X"].shape[-1])),
                "r": torch.mean(batch["X"], dim=0).reshape(1,-1)
              }
          ),
          "prior_categorical": UniformDistribution(
              {
                "a": torch.zeros((1, K)),
                "b": torch.ones((1, K))
              }
          ),
          "posterior_latent": NormalDistribution(
              {
              "mu": torch.zeros((1, 20)),
              "std": torch.ones((1, 20))
              }
          ),
        }
        for K in [2, 4]
    ]
)

In [27]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 50
BETA_KL = 0.5
WARMUP = None
PATIENCE = 5
TOL = 5e-3

In [30]:
model, losses, all_parts, clusters, all_betas = training_mvae(
    dataloader=train_loader,
    val_dataloader=val_loader,
    model=model,
    optimizer=optimizer,
    epochs=EPOCHS,
    beta_kl=BETA_KL,
    patience=PATIENCE,
    tol=TOL,
    model_type=1
)

TRAINING:   0%|          | 0/50 [00:29<?, ?it/s]

[tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]), tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])]


RuntimeError: Tensors must have same number of dimensions: got 3 and 2